In [1]:
#!/usr/bin/python
'''
Evaluate how well solutions match when run multiple times, with varying sample size
'''
import sys
import os
import fnmatch
import numpy as np
from metient.util.vertex_labeling_util import LabeledTree
from metient.util.globals import *
from metient.util import eval_util as eutil

import re
import pickle

REPO_DIR = os.path.join(os.getcwd(), "../../")

DATE = "multirun_07052024"
PARAMS = "calibrate_wip_solvepoly"

# plotting
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.5)

sns.set_style("whitegrid")
sns.set_style("ticks")
sns.despine()
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
pc_map = {'mS':0, 'pS': 1, 'mM': 2, 'pM': 3, 'mR': 4, 'pR': 5}

from statannot import add_stat_annotation


/lila/home/koyyald/mambaforge/envs/met/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


<Figure size 640x480 with 0 Axes>

In [2]:
import gzip

metient_results_dir = '/data/morrisq/divyak/data/metient_prediction_results/'

def get_num_mut_trees(mut_tree_fn):
    with open(mut_tree_fn, 'r') as f:
        # look for line w/ "3 #trees" as an example
        for line in f:
            if  "#trees" in line:
                return int(line.strip().split()[0])    

def get_pckl_info(bs, run, site, mig_type, seed, tree_num, topk):
    prediction_dir = os.path.join(metient_results_dir, f'predictions_batch_exp_bs{bs}_{PARAMS}_r{run}_{DATE}')
    predicted_site_mig_type_data_dir = os.path.join(prediction_dir, site, mig_type)
    
    with gzip.open(os.path.join(predicted_site_mig_type_data_dir, f"tree{tree_num}_seed{seed}_calibrate.pkl.gz"), "rb") as f:
        pckl = pickle.load(f)
    Vs = pckl[OUT_LABElING_KEY]
    k = topk if len(Vs) >= topk else len(Vs)
    Vs = Vs[:k]
    As = pckl[OUT_ADJ_KEY][:k]
    losses = pckl[OUT_LOSSES_KEY][:k]
    return Vs, As, losses

def get_rank_to_percent_runs(solution_to_losses, solution_to_ranks, run_num_to_solutions):
    # get the avg loss across runs for each solution
    for solution in solution_to_losses:
        solution_to_losses[solution] = sum(solution_to_losses[solution])/len(solution_to_losses[solution])
        solution_to_ranks[solution] = sum(solution_to_ranks[solution])/len(solution_to_ranks[solution])
    solution_losses_list = [(k,v) for k,v in solution_to_losses.items()]
    solution_ranks_list = [(k,v) for k,v in solution_to_ranks.items()]

#     print("unsorted solution_losses_list", solution_losses_list)
#     print("unsorted solution_ranks_list", solution_ranks_list)

    solution_losses_list = sorted(solution_losses_list, key=lambda x: x[1])
    solution_ranks_list = sorted(solution_ranks_list, key=lambda x: x[1])

#     print("sorted solution_losses_list", solution_losses_list)
#     print("sorted solution_ranks_list", solution_ranks_list)

    rank_to_percent_runs = {}
    # What % of runs get the solution as a function of the rank on the joined list?
    #for rank, (soln,_) in enumerate(solution_losses_list):
    for rank, (soln,_) in enumerate(solution_ranks_list):
        rank_to_percent_runs[rank] = []
        for run in run_num_to_solutions:
            if soln in run_num_to_solutions[run]:
                rank_to_percent_runs[rank].append(run)
    print(rank_to_percent_runs)
    rank_to_percent_runs = {rank:(len(rank_to_percent_runs[rank])/num_runs) for rank in rank_to_percent_runs}
    return rank_to_percent_runs

In [3]:
from tqdm import tqdm
import torch

topk = 5 # how many of the top solutions to check for 
num_runs = 5

sites = ["m8", "m5"]
mig_types = ["M", "mS", "R", "S"]
batch_sizes = [64, 256, 1024]

print(f"matching top {topk} solutions")

m5_bs_to_rank_to_percentages = {}
m8_bs_to_rank_to_percentages = {}
all_sites_bs_to_rank_to_percentages = {}

sims_dir = os.path.join(REPO_DIR, "data/machina_sims")

for bs in batch_sizes:
    print("batch size:", bs)
    m5_bs_to_rank_to_percentages[bs] = {}
    m8_bs_to_rank_to_percentages[bs] = {}
    all_sites_bs_to_rank_to_percentages[bs] = {}
    for site in sites:
        for mig_type in mig_types:
            # Get all seeds for mig_type + site combo
            site_mig_type_dir = os.path.join(sims_dir, site, mig_type)
            seeds = fnmatch.filter(os.listdir(site_mig_type_dir), 'clustering_observed_seed*.txt')
            seeds = [s.replace(".txt", "").replace("clustering_observed_seed", "") for s in seeds]
            for seed in tqdm(seeds):
                # Get all the clone trees for this seed
                num_trees = get_num_mut_trees(os.path.join(sims_dir, f"{site}_mut_trees", f"mut_trees_{mig_type}_seed{seed}.txt"))
                num_trees = 1
                for tree_num in range(num_trees):
                    # across all the runs for this mig_type + site + seed + clone tree combo, 
                    # get the union of all solutions
                    solution_to_losses, solution_to_ranks = {}, {}
                    run_num_to_solutions = {}
                    for run in range(1,num_runs+1):
                        Vs, As, losses = get_pckl_info(bs, run, site, mig_type, seed, tree_num, topk)
                        run_num_to_solutions[run] = []
                        print(site, mig_type, seed, "tree", tree_num, "run", run)
                        for rank,(V,A,loss) in enumerate(zip(Vs, As, losses)):
                            V = torch.tensor(V)
                            A = torch.tensor(A)
                            tree = LabeledTree(A, V)                            
                            if tree not in solution_to_losses:
                                solution_to_losses[tree] = []
                                solution_to_ranks[tree] = []
                            solution_to_losses[tree].append(loss)
                            solution_to_ranks[tree].append(rank)
                            run_num_to_solutions[run].append(tree)
                    rank_to_percent_runs = get_rank_to_percent_runs(solution_to_losses, solution_to_ranks, run_num_to_solutions)
                    #print(rank_to_percent_runs)
                    bs_to_rank_to_percentages = m5_bs_to_rank_to_percentages if site == "m5" else m8_bs_to_rank_to_percentages
                    for rank in rank_to_percent_runs:
                        if rank not in bs_to_rank_to_percentages[bs]:
                            bs_to_rank_to_percentages[bs][rank] = []
                            all_sites_bs_to_rank_to_percentages[bs][rank] = []
                        bs_to_rank_to_percentages[bs][rank].append(rank_to_percent_runs[rank])
                        all_sites_bs_to_rank_to_percentages[bs][rank].append(rank_to_percent_runs[rank])



 10%|█         | 1/10 [00:00<00:01,  8.25it/s]

matching top 5 solutions
batch size: 64
m8 M 45 tree 0 run 1
m8 M 45 tree 0 run 2
m8 M 45 tree 0 run 3
m8 M 45 tree 0 run 4
m8 M 45 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [2, 3, 4], 2: [1, 2, 4, 5], 3: [2, 3, 5], 4: [1, 2], 5: [1], 6: [3], 7: [4], 8: [5], 9: [5]}
m8 M 7 tree 0 run 1
m8 M 7 tree 0 run 2


 20%|██        | 2/10 [00:00<00:01,  7.45it/s]

m8 M 7 tree 0 run 3
m8 M 7 tree 0 run 4
m8 M 7 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [2], 2: [1, 2, 3, 4], 3: [1], 4: [2, 3, 4, 5], 5: [5], 6: [1, 5], 7: [3], 8: [2, 4], 9: [1, 5], 10: [3, 4]}
m8 M 35 tree 0 run 1
m8 M 35 tree 0 run 2
m8 M 35 tree 0 run 3
m8 M 35 tree 0 run 4
m8 M 35 tree 0 run 5


 50%|█████     | 5/10 [00:00<00:00,  9.19it/s]

...

 10%|█         | 1/10 [00:00<00:01,  8.55it/s]

m5 R 247 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5], 2: [1, 2, 3, 4, 5], 3: [1, 2, 3, 4, 5], 4: [1, 2, 3, 4, 5]}
m5 R 955 tree 0 run 1
m5 R 955 tree 0 run 2
m5 R 955 tree 0 run 3
m5 R 955 tree 0 run 4
m5 R 955 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5], 2: [1, 2, 3, 4, 5], 3: [1, 2, 3, 4, 5], 4: [1, 2], 5: [4]}
m5 R 518 tree 0 run 1
m5 R 518 tree 0 run 2
m5 R 518 tree 0 run 3
m5 R 518 tree 0 run 4
m5 R 518 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3]}
m5 R 17 tree 0 run 1
m5 R 17 tree 0 run 2
m5 R 17 tree 0 run 3
m5 R 17 tree 0 run 4
m5 R 17 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5]}
m5 S 49 tree 0 run 1
m5 S 49 tree 0 run 2
m5 S 49 tree 0 run 3
m5 S 49 tree 0 run 4
m5 S 49 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5], 2: [1, 2, 3, 4, 5], 3: [1, 2, 3, 4, 5], 4: [1, 2, 5], 5: [3, 4]}
m5 S 35 tree 0 run 1
m5 S 35 tree 0 run 2
m5 S 35 tree 0 run 3


  0%|          | 0/10 [00:00<?, ?it/s]

m5 S 35 tree 0 run 4
m5 S 35 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5]}
m5 S 25 tree 0 run 1
m5 S 25 tree 0 run 2
m5 S 25 tree 0 run 3
m5 S 25 tree 0 run 4
m5 S 25 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 4, 5]}
m5 S 31 tree 0 run 1
m5 S 31 tree 0 run 2
m5 S 31 tree 0 run 3
m5 S 31 tree 0 run 4
m5 S 31 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5], 2: [1], 3: [1, 4, 5]}
m5 S 81 tree 0 run 1
m5 S 81 tree 0 run 2
m5 S 81 tree 0 run 3
m5 S 81 tree 0 run 4
m5 S 81 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [2, 3], 2: [1, 2, 3, 4, 5], 3: [1, 3, 4, 5], 4: [1, 2, 3, 4, 5], 5: [1, 2, 4, 5]}
m5 S 40 tree 0 run 1
m5 S 40 tree 0 run 2
m5 S 40 tree 0 run 3
m5 S 40 tree 0 run 4
m5 S 40 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3, 4, 5], 2: [1, 2, 3, 4, 5]}
m5 S 32 tree 0 run 1
m5 S 32 tree 0 run 2
m5 S 32 tree 0 run 3
m5 S 32 tree 0 run 4
m5 S 32 tree 0 run 5
{0: [1, 2, 3, 4, 5], 1: [1, 2, 3]}
m5 S 23 tree 0 run 1
m5 S 23 tree 0 run 2
m5 S 23 tree 0 run 3
m5 S 23 tree 0 run 4
m5 S 

FileNotFoundError: [Errno 2] No such file or directory: '/data/morrisq/divyak/data/metient_prediction_results/predictions_batch_exp_bs1024_calibrate_wip_solvepoly_r1_multirun_07052024/m8/M/tree0_seed45_calibrate.pkl.gz'

In [ ]:
import pandas as pd

m8_data = []
for bs in m8_bs_to_rank_to_percentages:
    for rank in m8_bs_to_rank_to_percentages[bs]:
        avg = 100*(sum(m8_bs_to_rank_to_percentages[bs][rank])/len(m8_bs_to_rank_to_percentages[bs][rank]))
        m8_data.append([rank+1, f"Metient-{bs*4}", avg])
m8_df = pd.DataFrame(m8_data, columns=["Rank", "Method", "% runs"])
m8_df

In [ ]:
m5_data = []
for bs in m5_bs_to_rank_to_percentages:
    for rank in m5_bs_to_rank_to_percentages[bs]:
        avg = 100*(sum(m5_bs_to_rank_to_percentages[bs][rank])/len(m5_bs_to_rank_to_percentages[bs][rank]))
        m5_data.append([rank+1, f"Metient-{bs*4}", avg])
m5_df = pd.DataFrame(m5_data, columns=["Rank", "Method", "% runs"])
m5_df

In [ ]:
all_sites_data = []
for bs in all_sites_bs_to_rank_to_percentages:
    for rank in all_sites_bs_to_rank_to_percentages[bs]:
        avg = 100*(sum(all_sites_bs_to_rank_to_percentages[bs][rank])/len(all_sites_bs_to_rank_to_percentages[bs][rank]))
        all_sites_data.append([rank+1, f"Metient-{bs*4}", avg])
all_sites_df = pd.DataFrame(all_sites_data, columns=["Rank", "Method", "% runs"])
all_sites_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
colors = sns.color_palette("crest")
colors = colors[:len(batch_sizes)]
print(colors)
sns.set_palette(sns.color_palette(colors))

for name,df in zip(["m5", "m8"], [m5_df, m8_df]):
    with sns.plotting_context("notebook", font_scale=1.1):
        g = sns.relplot(
            data=df, x="Rank", y="% runs",
            height=4, aspect=1.0, dashes=False, linewidth=2.5,
            hue="Method", style="Method", kind="line", legend=True,
        )

        (g.set_axis_labels("Rank of solution", "% of runs that solution is found")
          .set_titles("boop")
          .tight_layout(w_pad=0))
        plt.xlim(1,16)
        #sns.move_legend(g, "upper right")
        plt.show()
        

        fig = g.fig
        #out_name = ("_").join(y_labels[i].split(" ") + x.split(" "))
        fig.savefig(os.path.join(REPO_DIR, "test/output_plots", f"{name}_percent_solns_found_{PARAMS}_{DATE}.png"), dpi=600) 


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
colors = sns.color_palette("crest")
colors = colors[:len(batch_sizes)]
print(colors)
sns.set(style='ticks', font="Helvetica", rc={'axes.labelsize': 10, 'xtick.labelsize': 8, 'ytick.labelsize': 8, 'axes.linewidth': 1.1})
sns.set_palette(sns.color_palette(colors))
plt.figure(figsize=(2, 2), dpi=200)

with sns.plotting_context("notebook", font_scale=1.2):
    g = sns.relplot(
        data=all_sites_df, x="Rank", y="% runs",
        height=4, aspect=1.0, dashes=False, linewidth=2.5,
        hue="Method", style="Method", kind="line",  legend=False,
    )

    (g.set_axis_labels("Rank of solution", "% of runs that solution is found")
      .set_titles("boop")
      .tight_layout(w_pad=0))
    plt.xlim(1,16)
    plt.show()


    fig = g.fig
    fig.savefig(os.path.join(REPO_DIR, "test/output_plots", f"all_sites_percent_solns_found_{PARAMS}_{DATE}.png"), dpi=600) 
